#### Code Reference: Python Machine Learning, 3rd Ed.

# Modeling Sequential Data Using Recurrent Neural Networks (part 2/2)

In [1]:
from IPython.display import Image

### Preprocessing the dataset

In [2]:
# 코랩에서 실행할 경우 다음 코드를 실행해 주세요.
!wget https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch16/1268-0.txt

--2023-05-08 11:13:51--  https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch16/1268-0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1171600 (1.1M) [text/plain]
Saving to: ‘1268-0.txt’

1268-0.txt          100%[===================>]   1.12M  --.-KB/s    in 0.07s   

2023-05-08 11:13:52 (15.7 MB/s) - ‘1268-0.txt’ saved [1171600/1171600]



In [3]:
import numpy as np


## 텍스트 읽고 전처리하기
with open('1268-0.txt', 'r', encoding='UTF8') as fp:
    text=fp.read()
    
start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')
print(start_indx, end_indx)

text = text[start_indx:end_indx]
char_set = set(text)
print('전체 길이:', len(text))
print('고유한 문자:', len(char_set))

567 1112917
전체 길이: 1112350
고유한 문자: 80


In [4]:
chars_sorted = sorted(char_set)
char2int = {ch:i for i,ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)

text_encoded = np.array(
    [char2int[ch] for ch in text],
    dtype=np.int32)

print('인코딩된 텍스트 크기: ', text_encoded.shape)

print(text[:15], '     == 인코딩 ==> ', text_encoded[:15])
print(text_encoded[15:21], ' == 디코딩 ==> ', ''.join(char_array[text_encoded[15:21]]))

인코딩된 텍스트 크기:  (1112350,)
THE MYSTERIOUS       == 인코딩 ==>  [44 32 29  1 37 48 43 44 29 42 33 39 45 43  1]
[33 43 36 25 38 28]  == 디코딩 ==>  ISLAND


In [5]:
import tensorflow as tf


ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for ex in ds_text_encoded.take(5):
    print('{} -> {}'.format(ex.numpy(), char_array[ex.numpy()]))

44 -> T
32 -> H
29 -> E
1 ->  
37 -> M


In [6]:
seq_length = 40
chunk_size = seq_length + 1

ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

## inspection:
for seq in ds_chunks.take(1):
    input_seq = seq[:seq_length].numpy()
    target = seq[seq_length].numpy()
    print(input_seq, ' -> ', target)
    print(repr(''.join(char_array[input_seq])), 
          ' -> ', repr(''.join(char_array[target])))

[44 32 29  1 37 48 43 44 29 42 33 39 45 43  1 33 43 36 25 38 28  1  6  6
  6  0  0  0  0  0 40 67 64 53 70 52 54 53  1 51]  ->  74
'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'  ->  'y'


In [7]:
## x & y를 나누기 위한 함수를 정의합니다
def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)

## 확인:
for example in ds_sequences.take(2):
    print('입력 (x):', repr(''.join(char_array[example[0].numpy()])))
    print('타깃 (y):', repr(''.join(char_array[example[1].numpy()])))
    print()

입력 (x): 'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'
타깃 (y): 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by'

입력 (x): ' Anthony Matonak, and Trevor Carlson\n\n\n\n'
타깃 (y): 'Anthony Matonak, and Trevor Carlson\n\n\n\n\n'



In [8]:
# 배치 크기
BATCH_SIZE = 64
BUFFER_SIZE = 10000

tf.random.set_seed(1)
ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)# drop_remainder=True)

ds

<_BatchDataset element_spec=(TensorSpec(shape=(None, 40), dtype=tf.int32, name=None), TensorSpec(shape=(None, 40), dtype=tf.int32, name=None))>

### Building a character-level RNN model

In [9]:
def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(
            rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

tf.random.set_seed(1)

model = build_model(
    vocab_size = charset_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         20480     
                                                                 
 lstm (LSTM)                 (None, None, 512)         1574912   
                                                                 
 dense (Dense)               (None, None, 80)          41040     
                                                                 
Total params: 1,636,432
Trainable params: 1,636,432
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True
    ))

model.fit(ds, epochs=20)

Epoch 1/20
424/424 [==============================] - 23s 24ms/step - loss: 2.3276
Epoch 2/20
424/424 [==============================] - 8s 17ms/step - loss: 1.7392
Epoch 3/20
424/424 [==============================] - 6s 13ms/step - loss: 1.5369
Epoch 4/20
424/424 [==============================] - 6s 13ms/step - loss: 1.4208
Epoch 5/20
424/424 [==============================] - 6s 13ms/step - loss: 1.3476
Epoch 6/20
424/424 [==============================] - 8s 16ms/step - loss: 1.2963
Epoch 7/20
424/424 [==============================] - 7s 14ms/step - loss: 1.2570
Epoch 8/20
424/424 [==============================] - 9s 16ms/step - loss: 1.2254
Epoch 9/20
424/424 [==============================] - 8s 17ms/step - loss: 1.1998
Epoch 10/20
424/424 [==============================] - 7s 13ms/step - loss: 1.1768
Epoch 11/20
424/424 [==============================] - 7s 16ms/step - loss: 1.1563
Epoch 12/20
424/424 [==============================] - 6s 13ms/step - loss: 1.1376
Epoch 13/20


### Evaluation phase: generating new text passages

In [11]:
tf.random.set_seed(1)

logits = [[1.0, 1.0, 1.0]]
print('확률:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10)
tf.print(samples.numpy())

확률: [0.33333334 0.33333334 0.33333334]
array([[1, 2, 0, 1, 0, 1, 1, 2, 1, 1]])


In [12]:
tf.random.set_seed(1)

logits = [[1.0, 1.0, 3.0]]
print('확률:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10)
tf.print(samples.numpy())

확률: [0.10650698 0.10650698 0.78698605]
array([[2, 2, 0, 2, 2, 2, 2, 2, 1, 2]])


In [13]:
def sample(model, starting_str, 
           len_generated_text=500, 
           max_input_length=40,
           scale_factor=1.0):
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))

    generated_str = starting_str

    model.reset_states()
    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)

        scaled_logits = logits * scale_factor
        new_char_indx = tf.random.categorical(
            scaled_logits, num_samples=1)
        
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()    

        generated_str += str(char_array[new_char_indx])
        
        new_char_indx = tf.expand_dims([new_char_indx], 0)
        encoded_input = tf.concat(
            [encoded_input, new_char_indx],
            axis=1)
        encoded_input = encoded_input[:, -max_input_length:]

    return generated_str

tf.random.set_seed(1)
print(sample(model, starting_str='The island'))

The island was egglopily this killed him? They now at a ciffes. Than
this wart of madmlost on the opening. The sea had been very project; the most impression here, were swenting to inaccess to eat the respect and toosphere.

The heat they were soon delish by the perimilar or was performantably falling in the ground to look anxile, and broken
of feelings which he was, that the bare had been decided with his companions, who mangered in any extremity of certain
empty weighers and replowed that some situation


* **Predictability vs. randomness**

In [14]:
logits = np.array([[1.0, 1.0, 3.0]])

print('스케일 조정 전의 확률: ', tf.math.softmax(logits).numpy()[0])

print('0.5배 조정 후 확률:  ', tf.math.softmax(0.5*logits).numpy()[0])

print('0.1배 조정 후 확률:  ', tf.math.softmax(0.1*logits).numpy()[0])

스케일 조정 전의 확률:  [0.10650698 0.10650698 0.78698604]
0.5배 조정 후 확률:   [0.21194156 0.21194156 0.57611688]
0.1배 조정 후 확률:   [0.31042377 0.31042377 0.37915245]


In [15]:
tf.random.set_seed(1)
print(sample(model, starting_str='The island', 
             scale_factor=2.0))

The island was explored the darkness was strewn with some from the neighborhood of the island was the position of the coast of the horizon. The sailor was stretched on
the shore, and a large shore of the vessel was struck by the heat of the southern planks of the earth. The sailor was not already passed them as if he left his clothes, which was seen somewhat
regard near the shore, the colonists had not been able to find out the explosion, which was explored the summit of the water to the forest, and Ayrto


In [16]:
tf.random.set_seed(1)
print(sample(model, starting_str='The island', 
             scale_factor=0.5))

The island had egblyin houl. No! nwelvel? ” would passed
if Chrotep them, I
rejoin,--”

“My but dot?” as favocears to enobes
abodabh, pa;
unfortunates yourse heart; Ayrton, however,” added Hehardion, proloeved
up, liste,
rassed
barquctic speedy.
Fredel frame, roking agotogoo and pe’s,! Therefore, nized it in genelagtill relish lineusticscbod.
A feells?” exclaimed Pencroft. Do faror
cry--thirty.”
 Ovember, but isiluted we
guily obmure and extremity verce, lenwire-field. They swarmed ieds, Harding JulyA9us,
